In [1]:
import tifffile
import pandas as pd
import joblib as jb
import numpy as np
import matplotlib.pyplot as plt
import ee
import geemap
from pathlib import Path

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
collection = ee.ImageCollection("COPERNICUS/S2_SR")

In [4]:
feature = Map.draw_last_feature
roi = feature.geometry()

In [5]:
out_dir = './download/'

In [9]:
collection_roi = collection.filterBounds(roi).filterDate('2018-01-01', '2020-01-01').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

In [25]:
filtered = collection_roi.filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 20)

In [26]:
count_filtered = filtered.size()
print('Count: ', str(count_filtered.getInfo())+'\n')

Count:  31



In [ ]:
collection_roi.first().getInfo()['bands']

In [11]:
count = collection_roi.size()
print('Count: ', str(count.getInfo())+'\n')

Count:  153



# Download filtered collection data

In [27]:
geemap.ee_export_image_collection(filtered, out_dir=out_dir, scale=25, file_per_band=True, region=roi)

Total number of images: 31

Exporting 1/31: 20181216T132221_20181216T132224_T22JGR.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/satellite_bathymetry/classification_notebooks/download


Exporting 2/31: 20190115T132231_20190115T132229_T22JGS.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/satellite_bathymetry/classification_notebooks/download


Exporting 3/31: 20190130T132239_20190130T132233_T22JGR.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/satellite_bathymetry/classification_notebooks/download


Exporting 4/31: 20190130T132239_20190130T132233_T22JGS.tif
Generating URL ...
Please wait ...
Data downloaded to /home/vinicius/Área de Trabalho/Faculdade/2020_1/TCC/satellite_bathymetry/classification_notebooks/download


Exporting 5/31: 20190209T132239_20190209T132233_T22JGR.tif
Generating URL ...
Pl

In [ ]:
geemap.ee_export_image_collection(collection_roi, out_dir=out_dir, scale=25, file_per_band=True, region=roi)

In [44]:
# Check images
from glob import glob
down_sufix = 'JGR'
upper_sufix = 'JGS'

images_dir = './download/best_images/'
down_images_list = [e for e in glob(f'{images_dir}*') if down_sufix in e]
up_images_list = [e for e in glob(f'{images_dir}*') if upper_sufix in e]

In [45]:
len(down_images_list), len(up_images_list) # why different? 

(104, 112)

In [54]:
sorted_down = sorted(down_images_list)
sorted_upper = sorted(up_images_list)

In [71]:
sorted_upper

['./download/best_images/20190130T132239_20190130T132233_T22JGS.B1.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B2.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B3.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B4.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B5.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B6.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B7.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGS.B8.tif',
 './download/best_images/20190209T132239_20190209T132233_T22JGS.B1.tif',
 './download/best_images/20190209T132239_20190209T132233_T22JGS.B2.tif',
 './download/best_images/20190209T132239_20190209T132233_T22JGS.B3.tif',
 './download/best_images/20190209T132239_20190209T132233_T22JGS.B4.tif',
 './download/best_images/20190209T132239_20190209T132233_T22JGS.B5.tif',
 './download/best_images/20190209T132239_20190209T1

In [72]:
sorted_down

['./download/best_images/20181216T132221_20181216T132224_T22JGR.B1.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B2.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B3.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B4.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B5.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B6.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B7.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B8.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B1.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B2.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B3.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B4.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B5.tif',
 './download/best_images/20190130T132239_20190130T1

In [63]:
sorted_dates_down = [e.split('/')[3].split('_T22JGR')[0] for e in sorted_down]
sorted_dates_up = [e.split('/')[3].split('_T22JGS')[0] for e in sorted_upper]

In [64]:
sorted_dates_down

['20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20181216T132221_20181216T132224',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190130T132239_20190130T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190209T132239_20190209T132233',
 '20190219T132229_20190219T132637',
 '20190219T132229_20190219T132637',
 '20190219T132229_20190219T132637',
 '20190219T132229_20190219T1

In [68]:
sorted_dates_down[0]

'20181216T132221_20181216T132224'

In [69]:
sorted_dates_up[0]

'20190130T132239_20190130T132233'

In [66]:
for i in range(104):
    is_equal = (sorted_dates_down[i] == sorted_dates_up[i])
    print(is_equal)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [49]:
a

In [56]:
sorted_down

['./download/best_images/20181216T132221_20181216T132224_T22JGR.B1.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B2.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B3.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B4.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B5.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B6.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B7.tif',
 './download/best_images/20181216T132221_20181216T132224_T22JGR.B8.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B1.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B2.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B3.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B4.tif',
 './download/best_images/20190130T132239_20190130T132233_T22JGR.B5.tif',
 './download/best_images/20190130T132239_20190130T1